# Reconstruct ME-model

In [ ]:
from IPython.display import display, HTML, Math, Markdown
display(HTML("<style>.container { width:95% !important; }</style>"))

%load_ext autoreload
%autoreload 2

In [ ]:
from coralme.builder.main import MEBuilder, MEReconstruction, METroubleshooter
from coralme.builder.curation import MECurator, MEManualCuration
import coralme
import pandas
pandas.set_option('display.max_colwidth', None)

In [ ]:
org = 'pputidame'
organism = './{:s}/organism.json'.format(org)
inputs = './{:s}/input.json'.format(org)

In [ ]:
import cobra
medium = cobra.io.load_json_model("./pputidame/inputs/m_model.json").medium
def enforce_medium(model,medium):
    for r in model.query("^EX_"):
        if r.id in medium:
            print("Enforcing " + r.id)
            r.lower_bound = -medium[r.id]
        else:
            print("Closing " + r.id)
            r.lower_bound = 0

### Full

In [ ]:
builder = MEBuilder(*[organism, inputs])
builder.configuration['run_bbh_blast'] = False

In [ ]:
builder.generate_files(overwrite=True)

In [ ]:
builder.build_me_model(overwrite=False)

In [ ]:
enforce_medium(builder.me_model,medium)
builder.me_model.reactions.EX_glc__D_e.bounds = (-1000,1000)

In [ ]:
for r in builder.me_model.reactions:
    if not isinstance(r,coralme.core.reaction.MetabolicReaction):
        continue
    r.keff = 35
    r.update()

In [ ]:
builder.troubleshoot(growth_key_and_value = { builder.me_model.mu : 0.001 }, guesses = set(["pg161_p", "4fe4s_c","dpm_c"]))

In [ ]:
# Save the model in quick access directory
coralme.io.json.save_json_me_model(builder.me_model, './model/iPpu1676-ME.json')
coralme.io.pickle.save_pickle_me_model(builder.me_model, './model/iPpu1676-ME.pkl')